<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [4]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [5]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [6]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches"

# Send a GET request to the page
response = requests.get(url)
response.raise_for_status()  # Check that the request was successful

# Assign the response content to an object
page_content = response.content

# Parse the HTML content of the page with BeautifulSoup
soup = BeautifulSoup(page_content, 'html.parser')

# Find the table that contains the launch records
# You may need to inspect the HTML structure of the page to find the correct table
# For this example, we assume it's the first table on the page
table = soup.find_all('table', {'class': 'wikitable'})[0]

# Extract the headers
headers = []
for header in table.find_all('th'):
    headers.append(header.text.strip())

# Extract the rows
rows = []
for row in table.find_all('tr'):
    cells = row.find_all('td')
    if len(cells) > 0:
        rows.append([cell.text.strip() for cell in cells])

# Ensure the number of columns in each row matches the number of headers
max_columns = len(headers)
adjusted_rows = []
for row in rows:
    if len(row) == max_columns:
        adjusted_rows.append(row)
    elif len(row) < max_columns:
        # Fill in missing columns with empty strings
        adjusted_rows.append(row + [''] * (max_columns - len(row)))

# Create a DataFrame
df = pd.DataFrame(adjusted_rows, columns=headers)

# Display the first few rows of the DataFrame
print(df.head())

# Save the DataFrame to a CSV file
df.to_csv('falcon9_launch_records.csv', index=False)

                                          Flight No.  \
0                            3 January 202314:56[22]   
1  Dedicated SmallSat Rideshare mission to Sun-sy...   
2                           10 January 202304:50[28]   
3  Following the Russian invasion of Ukraine, One...   
4                           15 January 202322:56[34]   

             Date andtime (UTC) Version,booster[c]  \
0          F9 B5 ♺ B1060.15[23]       CCSFS,SLC-40   
1                                                    
2               F9 B5 ♺ B1076.2       CCSFS,SLC-40   
3                                                    
4  Falcon Heavy B5 B1070 (core)         KSC,LC-39A   

                                          Launchsite            Payload[d]  \
0   Transporter-6: (115 payloads Smallsat Rideshare)            Unknown[e]   
1                                                                            
2  OneWeb Flight #16 / SpaceX Flight 2[29][30] (4...  6,000 kg (13,000 lb)   
3                         

Create a `BeautifulSoup` object from the HTML `response`


In [8]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
import requests
from bs4 import BeautifulSoup

# URL of the Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches"

# Send a GET request to the page
response = requests.get(url)
response.raise_for_status()  # Check that the request was successful

# Use BeautifulSoup to create a BeautifulSoup object from the response text content
soup = BeautifulSoup(response.text, 'html.parser')

# Now you can use the 'soup' object to find and extract data from the HTML
print(soup.prettify()[:500])  # Print the first 500 characters of the prettified HTML to verify

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-enabled vector-feature-appearance-pinned-clien


Print the page title to verify if the `BeautifulSoup` object was created properly 


In [9]:
# Use soup.title attribute
import requests
from bs4 import BeautifulSoup

# URL of the Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches"

# Send a GET request to the page
response = requests.get(url)
response.raise_for_status()  # Check that the request was successful

# Use BeautifulSoup to create a BeautifulSoup object from the response text content
soup = BeautifulSoup(response.text, 'html.parser')

# Get the title of the page
page_title = soup.title.string

print("Page Title:", page_title)

Page Title: List of Falcon 9 and Falcon Heavy launches - Wikipedia


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [10]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
import requests
from bs4 import BeautifulSoup

# URL of the Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches"

# Send a GET request to the page
response = requests.get(url)
response.raise_for_status()  # Check that the request was successful

# Use BeautifulSoup to create a BeautifulSoup object from the response text content
soup = BeautifulSoup(response.text, 'html.parser')

# Find all tables on the wiki page
tables = soup.find_all('table', {'class': 'wikitable'})

# Select the first table
first_table = tables[0]

# Extract all column/variable names from the table header
header_row = first_table.find('tr')
columns = [header.get_text(strip=True) for header in header_row.find_all('th')]

print("Column names:", columns)

Column names: ['Flight No.', 'Date andtime (UTC)', 'Version,booster[c]', 'Launchsite', 'Payload[d]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding']


Starting from the third table is our target table contains the actual launch records.


In [11]:
# Let's print the third table and check its content
first_launch_table = tables[2]
print(first_launch_table)

<table class="wikitable" style="width:100%; font-size:1.00em; line-height:1.5em;">
<tbody><tr class="is-sticky">
<th scope="col" style="width: 10%;">Date and time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>booster</a><sup class="reference" id="cite_ref-booster_27-2"><a href="#cite_note-booster-27">[c]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_28-2"><a href="#cite_note-Dragon-28">[d]</a></sup>
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th></tr>
<tr>
<td rowspan="2">8 July 2024<br/>03:10<sup class="reference" id="cite_ref-nextSFupcoming_378-1"><a href="#cite_note-nextSFupcoming-378">[370]</a></sup>
</td>
<td><a href="/wiki/Falcon_9_Block_5" title="Falcon 9 Block 5">F9 B5</a>
</td>
<td><a href="/wiki/Vandenberg_Space_Forc

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [12]:
column_names = []

# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names


Check the extracted column names


In [13]:
print(column_names)

[]


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [16]:
# Verify the column names
print(column_names)

# Create a dictionary with keys from the column names
launch_dict = dict.fromkeys(column_names)

# Check if the unwanted column name exists and delete it
unwanted_column = 'Date and time ( )'  
if unwanted_column in launch_dict:
    del launch_dict[unwanted_column]

# Initialize the launch_dict with each value to be an empty list
for key in launch_dict.keys():
    launch_dict[key] = []

# Add new columns and initialize them with empty lists
launch_dict['Version Booster'] = []
launch_dict['Booster landing'] = []
launch_dict['Date'] = []
launch_dict['Time'] = []
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []

# Print the dictionary to verify
print(launch_dict)

[]
{'Version Booster': [], 'Booster landing': [], 'Date': [], 'Time': [], 'Flight No.': [], 'Launch site': [], 'Payload': [], 'Payload mass': [], 'Orbit': [], 'Customer': [], 'Launch outcome': []}


Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [18]:
import requests
from bs4 import BeautifulSoup

# Send the HTTP request and get the HTML response
url = "https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches"
response = requests.get(url)

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Define functions (date_time, booster_version, get_mass, landing_status) here if not already defined

# Initialize launch_dict with all necessary keys
launch_dict = {
    'Flight No.': [], 
    'Date': [], 
    'Time': [], 
    'Version Booster': [], 
    'Launch site': [], 
    'Payload': [], 
    'Payload mass': [], 
    'Orbit': [], 
    'Customer': [], 
    'Launch outcome': [], 
    'Booster landing': []
}

extracted_row = 0

# Iterate over each table
for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    for rows in table.find_all("tr"):
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()
        else:
            flag = False
        row = rows.find_all('td')
        if flag:
            extracted_row += 1
            # Flight Number value
            launch_dict['Flight No.'].append(flight_number)

            # Date and Time values
            datatimelist = date_time(row[0])
            date = datatimelist[0].strip(',')
            time = datatimelist[1]
            launch_dict['Date'].append(date)
            launch_dict['Time'].append(time)

            # Booster version
            bv = booster_version(row[1])
            if not(bv):
                bv = row[1].a.string
            launch_dict['Version Booster'].append(bv)

            # Launch Site
            launch_site = row[2].a.string if row[2].a else None
            launch_dict['Launch site'].append(launch_site)

            # Payload
            payload = row[3].a.string if row[3].a else None
            launch_dict['Payload'].append(payload)

            # Payload Mass
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)

            # Orbit
            orbit = row[5].a.string if row[5].a else None
            launch_dict['Orbit'].append(orbit)

            # Customer
            customer = row[6].a.string if row[6].a else None
            launch_dict['Customer'].append(customer)

            # Launch outcome
            launch_outcome = list(row[7].strings)[0] if row[7].strings else None
            launch_dict['Launch outcome'].append(launch_outcome)

            # Booster landing
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)

# Print launch_dict to verify the values
print(launch_dict)

{'Flight No.': ['195', '196', '197', '198', '199', '200', '201', '202', '203', '204', '205', '206', '207', '208', '209', '210', '211', '212', '213', '214', '215', '216', '217', '218', '219', '220', '221', '222', '223', '224', '225', '226', '227', '228', '229', '230', '231', '232', '233', '234', '235', '236', '237', '238', '239', '240', '241', '242', '243', '244', '245', '246', '247', '248', '249', '250', '251', '252', '253', '254', '255', '256', '257', '258', '259', '260', '261', '262', '263', '264', '265', '266', '267', '268', '269', '270', '271', '272', '273', '274', '275', '276', '277', '278', '279', '280', '281', '282', '283', '284', '285', '286', '287', '288', '289', '290', '291', '292', '293', '294', '295', '296', '297', '298', '299', '300', '301', '302', '303', '304', '305', '306', '307', '308', '309', '310', '311', '312', '313', '314', '315', '316', '317', '318', '319', '320', '321', '322', '323', '324', '325', '326', '327', '328', '329', '330', '331', '332', '333', '334', '335

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [19]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

In [20]:
import pandas as pd

# Assuming launch_dict is already filled with data

# Convert the dictionary to a DataFrame
df = pd.DataFrame(launch_dict)

# Export the DataFrame to a CSV file
df.to_csv('spacex_web_scraped.csv', index=False)

print("Data exported to spacex_web_scraped.csv")

Data exported to spacex_web_scraped.csv


We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |


Copyright © 2021 IBM Corporation. All rights reserved.
